### Постановка задачи
Даны неповторяющиеся последовательности нуклеотидов (риды) длины l. Известно, что все подстроки генома длины l входят в данное множество ридов. Построить возможный геном.

#### Пояснение
Геном (как и его части − риды) является словом из 4-символьного алфавита {A,G,C,T}, где символы так же называются нуклеотидами. В реальности длины ридов находятся обычно в диапазоне 100-1000 нуклеотидов, а геном может содержать от 10^6 нуклеотидов у простейших организмов. При этом учёные могут получать информацию только о ридах (в силу размера последовательностей) физическим путем (метод секвенирования).

#### Решение
Решение этой задачи очень простое после решения предыдущей задачи. Построим граф G, где вершинами будут являться суффиксы и префиксы длины l−1 всех ридов. Получили подграф графа де Брюина (4,l−1) (подграф, так как в нём есть не обязательно все 4l−1 вершины), где каждому ребру соответствует рид. Найдём в нём эйлеров путь. Он существует, так как на геном было наложено условие о том, что все его подстроки длины l входят в наше множество ридов. Этот путь является возможным ответом. Очевидно, что единственно верный ответ (коим является реальный геном реального существа) получить можно не всегда, так как не всегда в графе есть единственный эйлеров путь.

##### Проще 

Во время сборки графа Де Брёйна риды разбиваются на более мелкие фрагменты заданного размера k. Затем k-меры используются в качестве узлов в сборке графа. Узлы, которые частично перекрываются (обычно k-1), затем соединяются ребром. Затем ассемблер строит последовательности на основе графа Де Брёйна.

In [54]:

from collections import defaultdict
import sys
import typing
from typing import List, Union
import random
import copy



In [11]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [37]:
#здесь строчку разбиваем на К-меры 
def kmers(string,k) -> str:
    for i in range(len(string) - k + 1):
        yield string[i:i+k]



In [44]:
#на входе граф записываем словарем: ключ-одна вершина, значения - вершины к которым есть ребро.
def Tour(start:str, graph:defaultdict, end: Union[str, None] =None) -> List[str]: 
    
    tour = [start] 

    if end is None:
        finish = start
    else:
        finish = end
    while True:
        if tour[-1] not in graph.keys():
            break

        nodes = graph[tour[-1]]
        if not nodes: #последовательность узлов-не эйлеров путь
            break
            
        tour.append(nodes.pop())
        if tour[-1] == finish: # эйлеров путь
            break
  #print(len(tour))
        offset = 0
        for i,step in enumerate(tour):
            try:
                nodes = graph[step]
                #print(nodes)
                if nodes: 
                    
                    tour_ = Tour(nodes.pop(), graph, step) #generate a subtour
                    i += offset
                    tour = tour[ : i + 1 ] + tour_ + tour[ i + 1 : ]
                    offset += len(tour_)
                    print("-----------")
                    

            except:
                #print("Oops")
                continue
            
        return tour



In [52]:
#на входе список с К-мерами с заданной длиной
def BuildDeBrujinGraph(reads: List[List[str]], k=25) -> defaultdict:
    graph = defaultdict(list)
    all_kmers = set()
    for read in reads:
        for kmer in kmers(read, k):
            head = kmer[:-1]
            tail = kmer[1:]
            graph[head].append(tail)

    return graph



In [53]:
def BuildGenomeFromDeBrujnGraph(graph:defaultdict):
    start = None
    for key in list(graph.keys()):
        if len(graph[key]) % 2 == 0 and len(graph[key]) != 0:
            start = key
            break
    print(f"START: {start}")
    tour = Tour(start, graph)
    return "".join([tour[0]] + [s[-1] for s in tour[1:]])